<a href="https://colab.research.google.com/github/AlexKnowsIt/Udemy_Deal_searcher/blob/master/MP_Textauswertung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from openai import OpenAI
import json
import time
import datetime
import openpyxl
import os
import re
import string
from openpyxl import load_workbook

folder_path = '/content/drive/My Drive/MP_Textauswertung'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

####
source_file_path = '/content/drive/MyDrive/Colab Notebooks/Test-Daten-4.xlsx'
####

working_file_path = os.path.join(folder_path, 'working_file.xlsx')
df = pd.read_excel(source_file_path, engine='openpyxl', sheet_name=0)

if os.path.exists(working_file_path):
    with pd.ExcelWriter(working_file_path, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
        df.to_excel(writer, sheet_name='New Sheet', index=False)
else:
    df.to_excel(working_file_path, index=False)

1. "source_file_path" zu dem Path ändern aus dem die Datei kommt
2. df = pd.read_excel(source_file_path, engine='openpyxl', **sheet_name=0**) -> 0 = erstes sheet in der Datei

In [ ]:
key_column = 'key' # Column name for the ID to limit going through certain columns
wait_time = 1 # wait time in seconds (to limit API use to not get rate limited)
openai_api_key = ''  # OpenAI API key

In [ ]:
def call_api_3(prompt):
    # Wait for a specified time before making the API call
    time.sleep(wait_time)
    client = OpenAI(api_key=openai_api_key)

    try:
        # Making the call to the GPT-3 API
        chat_completion = client.chat.completions.create(
            model= "gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )

        # Extracting the text response
        api_response = chat_completion.choices[0].message.content.strip() if chat_completion.choices else 'No response'

        # Preparing the response data
        response_data = {
            'prompt': prompt,
            'response': api_response
        }

    except Exception as e:
        # If there's an error, return the entire error message
        response_data = {
            'prompt': prompt,
            'response': str(e)
        }

    return response_data


def call_api_4(prompt):
    # Wait for a specified time before making the API call
    time.sleep(wait_time)

    # Initialize the OpenAI client with the provided API key
    client = OpenAI(api_key=openai_api_key)

    try:
        # Making the call to the GPT-4 API
        chat_completion = client.chat.completions.create(
            model="gpt-4-1106-preview",  # Updated model name for GPT-4
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )

        # Extracting the text response
        api_response = chat_completion.choices[0].message.content.strip() if chat_completion.choices else 'No response'

        # Preparing the response data
        response_data = {
            'prompt': prompt,
            'response': api_response
        }

    except Exception as e:
        # If there's an error, return the entire error message
        response_data = {
            'prompt': prompt,
            'response': str(e)
        }

    return response_data

def call_api_test(prompt):
    responses = [
        "test1; test2; test3",
        "test4; test2; test3",
        "test4; test5; test3",
        "test4; test5; test6"
    ]

    index = len(prompt) % 4
    api_response = responses[index]

    response_data = {
        'prompt': prompt,
        'response': api_response
    }

    return response_data

In [ ]:
def find_column_topics(key, context, subject, content_column, max_number=None):
    # Add a check for key_column existence
    if key_column not in df.columns:
        raise ValueError(f"Key column '{key_column}' does not exist in the DataFrame.")

    # API4 instruction - designed to be clear yet editable
    api4_instruction = f"Nennen die {subject} des Textes. Führen Sie sie durch Semikolon ';' getrennt auf. Deine Antwort besteht nur aus {subject}, die durch Semikolon ';' getrennt sind."

    # Step 1: Extract topics from the whole column using API4
    matching_rows = df[df[key_column] == key]  # Filter rows matching the key
    responses = matching_rows[content_column].tolist()[:max_number]
    question = content_column  # The content column is treated as the question

    whole_column_prompt = (
        f"{api4_instruction}\n\n"
        f"{question}: " + " ###### ".join(str(response) for response in responses) + " ######\n\n"
        f"{context}\n\n"
        f"{api4_instruction}"
    )
    api_result_whole = call_api_4(whole_column_prompt)  # Ensure this function is defined

    # Step 2: Populate the Topics List
    topics = api_result_whole['response'].split(';')
    topics = [topic.strip() for topic in topics if topic.strip()]

    processed_rows = []  # To store processed data

    # API3 instruction
    api3_instruction = f"Bestimme die {subject} aus der vorgegebenen Liste, die im Text vorkommen. Führe die {subject} durch ';' getrennt auf. Deine Antwort besteht nur aus {subject}, die durch Semikolon ';' getrennt sind."

    # Step 3: Iterate through the matching elements in the column using API3
    processed_count = 0  # Counter for the number of processed rows
    for _, row in matching_rows.iterrows():
        review = row[content_column]
        row_data = {
            'prompt': "Skipped",
            'response': "Skipped",
            'response_topics': "Skipped"
        }

        # Check if review is not a string or is an empty/whitespace string
        if not isinstance(review, str) or not review.strip():
            processed_rows.append(row_data)
            continue

        # Construct the prompt for API3
        topics_str = '; '.join(topics)
        full_prompt = (
            f"Frage: {question}\n"
            f"------ **ANFANG DES TEXTES** ------\n"
            f"{review}\n"
            f"------ **ENDE DES TEXTES** ------\n\n"
            f"{api3_instruction}\n"
            f"Bereits vorhandene Themen:\n{topics_str}\n\n"
            f"> "
        )

        api_result_row = call_api_3(full_prompt)  # Ensure this function is defined

        # Convert response topics from string to list
        response_topics = api_result_row['response'].split(';')
        response_topics = [
            topic.translate(str.maketrans('', '', string.punctuation)).lower().strip()
            for topic in response_topics if topic.strip()
        ]

        # Prepare row data for final DataFrame
        row_data = {
            'prompt': api_result_row['prompt'],
            'response': api_result_row['response'],
            'response_topics': response_topics
        }
        processed_rows.append(row_data)

        # Increment the processed row counter
        processed_count += 1
        if max_number is not None and processed_count >= max_number:
            break

    # Step 4: Assemble and Save DataFrame
    result_df = pd.DataFrame(processed_rows)

    # Add columns for each topic and mark presence in each row
    for topic in topics:
        # Ensure the column exists before applying the lambda function
        if 'response_topics' not in result_df.columns:
            result_df['response_topics'] = pd.Series([[] for _ in range(len(result_df))])
        result_df[f"topic: {topic}"] = result_df['response_topics'].apply(lambda x: 1 if topic.lower() in (t.lower() for t in x) else 0)

    # Generate a current datetime string for the sheet name
    current_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    cleaned_content_column = re.sub(r'[^\w\s-]', '', content_column)
    sheet_name = f"{cleaned_content_column} - {current_datetime}"

    # Save the processed data to an Excel file
    with pd.ExcelWriter(f'{folder_path}/working_file.xlsx', engine='openpyxl', mode='a') as writer:
        result_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Saved the results to {sheet_name} in {folder_path}")

    return result_df

In [ ]:
def append_to_excel(working_file_path, sheet_name, result_row):
    # Create a DataFrame from the result row
    new_data = pd.DataFrame([result_row])

    if not os.path.exists(working_file_path):
        # If the file doesn't exist, write new data to a new file
        with pd.ExcelWriter(working_file_path, engine='openpyxl') as writer:
            new_data.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        # If the file exists, read the existing data and append the new row
        with pd.ExcelWriter(working_file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            try:
                # Try to read the existing sheet
                existing_data = pd.read_excel(working_file_path, sheet_name=sheet_name)
                updated_data = pd.concat([existing_data, new_data], ignore_index=True)
            except ValueError:
                # If the sheet doesn't exist, just use the new data
                updated_data = new_data

            # Write/overwrite the sheet with the updated data
            updated_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
def column_summary(key, context, instruction, content_column, max_number = None):
    sheet_name = "Analyse - Ergebnisse"
    if key_column not in df.columns:
        raise ValueError(f"Key column '{key_column}' does not exist in the DataFrame.")

    # Filter rows matching the key
    matching_rows = df[df[key_column] == key]

    # Assemble the prompt
    header = content_column
    responses = matching_rows[content_column].tolist()[:max_number]  # Get the first 'max_number' responses from the filtered rows
    prompt = f"{context}\n{instruction}\n{header}: " + " ###### ".join(str(response) for response in responses) + " ######\n" + f"{instruction}"

    api_result = call_api_4(prompt)

    current_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Results
    result_row = {
        'Datetime': current_datetime,
        'content_column': content_column,
        'Inputs/Prompt': api_result['prompt'],
        'Output': api_result['response']
    }

    ### Apend goes here ###
    append_to_excel(working_file_path, sheet_name, result_row)

    print(f"Saved the results to {sheet_name} in {working_file_path}")
    return result_row

In [ ]:
def multi_column_summary(key, context, instruction, content_column, max_number=None):
    sheet_name = "Analyse - Ergebnisse"
    if key_column not in df.columns:
            raise ValueError(f"Key column '{key_column}' does not exist in the DataFrame.")
    matching_rows = df[df[key_column] == key]

    # Initialize the prompt with context and instruction
    prompt = f"{context}\n{instruction}\n######\n"

    # Append each column's responses to the prompt
    for col in content_column:
        column_header = col
        responses = matching_rows[col].dropna().tolist()[:max_number]  # Get non-NaN 'max_number' responses from the filtered rows
        column_responses = " ###\n".join([column_header] + [str(response) for response in responses]) # Include the header in responses
        prompt += f"{column_responses}\n###\n"

    # Add the final separator and instruction
    prompt += "######\n" + f"{instruction}"

        # Call the API
    api_result = call_api_4(prompt)

    # Prepare the results
    current_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Results
    result_row = {
        'Datetime': current_datetime,
        'content_column': '; '.join(content_column),  # Join all column names for the record
        'Inputs/Prompt': api_result['prompt'],
        'Output': api_result['response']
    }

    ### Append goes here ###
    append_to_excel(working_file_path, sheet_name, result_row)

    print(f"Saved the results to {sheet_name} in {working_file_path}")
    return result_row

In [ ]:
context1 = "FlowFlex Piping ist ein mittelständisches Unternehmen, das sich auf die Herstellung von industrietauglichen Rohrverbindern spezialisiert hat. Diese Verbinder finden breite Anwendung in den Rohrsystemen von großen Gebäuden, Fabriken und städtischen Wasserversorgungssystemen."
subject1 = "Hauptthemen"
content_column1 = "Welche Maßnahmen oder Initiativen könnten Ihrer Meinung nach die Mitarbeiterzufriedenheit und -bindung in Ihrem Unternehmen verbessern?"

result = find_column_topics(2, context1, subject1, content_column1)
print(result)

In [ ]:
context2 = "FlowFlex Piping ist ein mittelständisches Unternehmen, das sich auf die Herstellung von industrietauglichen Rohrverbindern spezialisiert hat. Diese Verbinder finden breite Anwendung in den Rohrsystemen von großen Gebäuden, Fabriken und städtischen Wasserversorgungssystemen."
instruction2 = "Dies sind Antworten auf eine Frage - was kann aus den Antworten abgeleitet werden?"
content_column2 = "Welche Maßnahmen oder Initiativen könnten Ihrer Meinung nach die Mitarbeiterzufriedenheit und -bindung in Ihrem Unternehmen verbessern?"

result = column_summary(2, context2, instruction2, content_column2, max_number = 5)
print(result)

In [ ]:
context3 = "FlowFlex Piping ist ein mittelständisches Unternehmen, das sich auf die Herstellung von industrietauglichen Rohrverbindern spezialisiert hat. Diese Verbinder finden breite Anwendung in den Rohrsystemen von großen Gebäuden, Fabriken und städtischen Wasserversorgungssystemen."
instruction3 = "Dies sind Antworten auf eine Reihe von Fragen - was sind die wichtigsten Punkte die daraus abgeleitet werden können?"
content_column3 = ["Welche Chancen und Herausforderungen erkennen Sie für ihr Unternehmen?", "Wie würden Sie die Unternehmenskultur und das Arbeitsumfeld in Ihrer aktuellen Position beschreiben?", "Welche Maßnahmen oder Initiativen könnten Ihrer Meinung nach die Mitarbeiterzufriedenheit und -bindung in Ihrem Unternehmen verbessern?"]  # List of column names to include in the prompt

result = multi_column_summary(2, context3, instruction3, content_column3, max_number=5)
print(result)